In [2]:
# Install matplotlib (usually comes with Anaconda/Jupyter, but good to ensure)
!pip install matplotlib --quiet

# Install numpy (usually comes with Anaconda/Jupyter, but good to ensure)
!pip install numpy --quiet

In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors 
import numpy as np
import os 

In [8]:
def load_graph_from_gml_file(graph_file, weight_attribute_name="weight"):
    # Check if the graph file exists
    if not os.path.exists(graph_file):
        print(f"Error: Graph file '{graph_file}' not found.")
        print(f"Please ensure '{graph_file}' is a valid path to your 'lesmis.gml' file.")
        print("You can typically find this file by searching for 'lesmis.gml network dataset'.")
        return # Exit the function if file is not found

    try:
        # Load network from GML file
        # igraph.Graph.Read_GML will automatically load edge attributes like 'value'
        # if they are present in the GML file.
        graph = ig.Graph.Read_GML(graph_file)
        
        # Check if the graph has the correct weight attribute name
        if weight_attribute_name not in graph.edge_attributes():
            print(f"Warning: Graph '{graph_file}' does not have a '{weight_attribute_name}' attribute. "
                  "Community detection will proceed without explicit weights, or if the algorithm "
                  "expects them, it might use default uniform weights.")
            # If no 'value' attribute, assign a default uniform weight for visualization purposes
            graph.es[weight_attribute_name] = 1 

        return graph


    except Exception as e:
        print(f"An error occurred while loading or processing the graph: {e}")
        return

def community_detection(graph, community_detection_method="multilevel", weight_attribute_name="weight"):
    if community_detection_method == "multilevel":
        return graph.community_multilevel(weights=weight_attribute_name if weight_attribute_name in graph.edge_attributes() else None)
    elif community_detection_method == "fastgreedy":
        return graph.community_fastgreedy(weights=weight_attribute_name if weight_attribute_name in graph.edge_attributes() else None).as_clustering()


# Functions useful to test community structure

In [4]:

def get_modularity_on_clustering(graph, community_detection_method="multilevel"):
    partition = community_detection(graph, community_detection_method, weight_attribute_name=None)
    return partition.modularity

def rewire(graph):
    num_randomizations = 500  # Number of randomized networks to generate
    modularity_random_networks = []
    
    num_swaps_for_randomization = graph.ecount()
    
    for i in range(num_randomizations):
        # G.rewire() modifies the graph in-place, so we must work on a copy.
        graph_random = graph.copy()
    
        graph_random.rewire(n=num_swaps_for_randomization)
    
        modularity_random_networks.append(get_modularity_on_clustering(graph_random))

    return modularity_random_networks

def plot_histogram(modularity_original, modularity_random_networks, graph_name="Karate Club Network"):
    plt.figure(figsize=(10, 6))
    plt.hist(modularity_random_networks, bins=30, alpha=0.7, color='lightgreen',
             edgecolor='black', label='Modularity of Randomized Networks')
    
    # Plot a vertical line for the original network's modularity
    plt.axvline(modularity_original, color='red', linestyle='dashed', linewidth=2,
                label=f'Original Network Modularity ({modularity_original:.4f})')
    
    plt.title(f'Modularity of Original vs. Randomized {graph_name} (igraph)')
    plt.xlabel('Modularity Score')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(axis='y', alpha=0.75)
    plt.tight_layout()
    plt.show()


def test_community_structure(graph, graph_name="Karate Club Network", community_detection_method="multilevel"):
    modularity_orig = get_modularity_on_clustering(graph, community_detection_method)
    modularity_random_networks = rewire(graph)
    plot_histogram(modularity_orig, modularity_random_networks, graph_name)
